In [1]:
# Подключение библиотек

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates

In [2]:
# Чтение необходимых файлов 
assessments = pd.read_csv('assessments.csv')
courses = pd.read_csv('courses.csv')
studentAssessment = pd.read_csv('studentAssessment.csv')
studentRegistration = pd.read_csv('studentRegistration.csv')

In [3]:
# Задание 1. 
# Сколько студентов успешно сдали только один курс?

In [4]:
# Соединяю таблички studentAssessment и assessments, чтобы получить информацию о типе экзаменов, которые сдавали студенты
merge_studentAssessment_assessments = pd.merge(studentAssessment, assessments, how='left',
                                     left_on=['id_assessment'],
                                     right_on=['id_assessment'])

In [5]:
# Создаю табличку, в которой отбираются студенты, 
# которые успешно сдали экзамен (тип экзамена - Exam, успешная сдача - оценка >=40).
# Далее группирую по id студентов, чтобы посчитать, сколько курсов каждый завершил с успешной сдачей экзамена
# и отбираю только тех, у кого количество курсов = 1.
student_exam = merge_studentAssessment_assessments.query("assessment_type =='Exam' and score >=40")\
                                    .groupby(['id_student'], as_index=False)\
                                    .agg({'id_assessment':'count'}).rename(columns={'id_assessment':'exam_count'})\
                                    .query("exam_count==1")

In [6]:
# считаю и вывожу количество студентов, успешно сдавших только 1 курс
students_count = student_exam.agg({'id_student':'count'})[-1]
print('Количестов студентов, успешно сдавших только 1 курс = ', students_count )

Количестов студентов, успешно сдавших только 1 курс =  3802


In [7]:
# Задание 2. 
# Выяви самый сложный и самый простой экзамен: найди курсы и экзамены в рамках курса, 
# которые обладают самой низкой и самой высокой завершаемостью*
# *завершаемость = кол-во успешных экзаменов / кол-во всех попыток сдать экзамен

In [8]:
# Сначала рассчитаем количество попыток сдать экзамен по каждому из предметов.
# Для этого выберем тип экзамена 'Exam', сгруппируем данные по типу предмета и посчитаем количество попыток сдать экзамен.
# Далее рассчитаем количество успешных попыток(score>=40) сдать экзамен. Для этого также выберем тип экзамена 'Exam'
# и количество баллов не менее 40 (score>=40). Сгруппируем данные по типу предмета и посчитаем количество удачных попыток.
# Соединим две получившиеся таблички по коду предмета для удобства ддальнейших преобразований.

all_tries = merge_studentAssessment_assessments.query("assessment_type =='Exam'").groupby(['code_module'], as_index=False)\
            .agg({'id_assessment':'count'})\
            .rename(columns={'id_assessment':'count_tries'})
succsess = merge_studentAssessment_assessments.query("assessment_type =='Exam' and score>=40")\
            .groupby(['code_module'], as_index=False).agg({'id_assessment':'count'})\
            .rename(columns={'id_assessment':'count_succsess'})
table_succsess_all = pd.merge (succsess, all_tries)

In [9]:
table_succsess_all

,code_module,count_succsess,count_tries
0,CCC,1683,1915
1,DDD,2709,3044


In [10]:
# Создаю колонку, в которой рассчитывается завершаемость каждого предмета 

table_succsess_all['completeness'] = round((100*table_succsess_all.count_succsess / table_succsess_all.count_tries),2)

In [11]:
# Максимальная завершаемость 
table_succsess_all.sort_values('completeness', ascending=False).head(1)

,code_module,count_succsess,count_tries,completeness
1,DDD,2709,3044,88.99


In [12]:
# Минимальная завершаемость 
table_succsess_all.sort_values('completeness').head(1)

,code_module,count_succsess,count_tries,completeness
0,CCC,1683,1915,87.89


In [13]:
# Задание 3.
# По каждому предмету определи средний срок сдачи экзаменов 
# (под сдачей понимаем последнее успешное прохождение экзамена студентом)

In [14]:
# Из таблички merge_studentAssessment_assessments, в которой находится информатиция о студентах и тапих экзаменов,
# которые они сдавали, необходимо выбрать тех, кто написал экзамен (assessment_type=='Exam') 
# и получил не менее 40 баллов (score>=40). Далее группируем по предмету и типу теста и счтитаем среднее количество дней, 
# которое студенты потратили на сдачу теста.

mean_time = merge_studentAssessment_assessments.query("assessment_type=='Exam' and score>=40")\
                                    .groupby(['code_module', 'assessment_type'], as_index=False)\
                                    .agg({'date_submitted':'mean'}).rename(columns={'date_submitted':'mean_submit_time'})
mean_time['mean_submit_time'] = round(mean_time['mean_submit_time'], 1)
mean_time

,code_module,assessment_type,mean_submit_time
0,CCC,Exam,239.3
1,DDD,Exam,238.0


In [15]:
# Задание 4.
# Выяви самые популярные предметы (ТОП-3) по количеству регистраций на них. 
# А также предметы с самым большим оттоком (ТОП-3)

In [16]:
# Чтобы найти самый популярный предмет по количеству регистраций на него, нужно табличку studentRegistration
# сгруппировать по предмету (code_module). Далее для каждой группы считаем количество регистраций и сортируем по убыванию
# количества регистраций. Выводим 3 самый популярных предмета по количеству регистраций.

TOP3_registrations = studentRegistration.groupby(['code_module'], as_index=False)\
                    .agg({'date_registration':'count'}).rename(columns={'date_registration':'number_of_reg'})\
                    .sort_values('number_of_reg', ascending=False).head(3)
TOP3_registrations

,code_module,number_of_reg
1,BBB,7900
5,FFF,7751
3,DDD,6257


In [17]:
# Чтобы найти предметы с самым большим оттоком, нужно в табличке studentRegistration
# выбрать все не пустые даты отмены регистрации (date_unregistration),  
# сгруппировать по предмету (code_module). Далее для каждой группы считаем количество отменненных регистраций 
# и сортируем по убыванию количества отмен регистраций.
# Выводим 3 самый популярных предмета по количеству отмен регистраций.

TOP3_unregistrations = studentRegistration.query("date_unregistration!='NaN'")\
                                        .groupby(['code_module'], as_index=False)\
                                        .agg({'date_unregistration':'count'})\
                                        .rename(columns={'date_unregistration':'number_of_unreg'})\
                                        .sort_values('number_of_unreg', ascending=False).head(3)
TOP3_unregistrations

,code_module,number_of_unreg
5,FFF,2380
1,BBB,2377
3,DDD,2235


In [18]:
# Задание 5.
# Используя pandas, в период с начала 2013 по конец 2014 выяви семестр с самой низкой 
# завершаемостью курсов и самыми долгими средними сроками сдачи курсов.

In [19]:
# Сначала рассчитаем количество попыток сдать экзамен по каждому из предметов в каждом семестре.
# Для этого выберем тип экзамена 'Exam', сгруппируем данные по семестру и типу предмета
# и  посчитаем количество попыток сдать экзамен.
# Далее рассчитаем количество успешных попыток(score>=40) сдать экзамен. Для этого также выберем тип экзамена 'Exam'
# и количество баллов не менее 40 (score>=40). Сгруппируем данные по семестру и  типу предмета 
# и посчитаем количество удачных попыток.
# Соединим две получившиеся таблички по семестру и коду предмета  для удобства дальнейших преобразований.

all_tries_sem = merge_studentAssessment_assessments.query("assessment_type =='Exam'")\
                                    .groupby(['code_presentation', 'code_module'], as_index=False)\
                                    .agg({'id_assessment':'count'})\
                                    .rename(columns={'id_assessment':'count_all_tries'})

succsess_sem = merge_studentAssessment_assessments.query("assessment_type =='Exam' and score >=40")\
                                    .groupby(['code_presentation', 'code_module'], as_index=False)\
                                    .agg({'id_assessment':'count'})\
                                    .rename(columns={'id_assessment':'count_succsess_tries'})

table_1 = pd.merge(all_tries_sem, succsess_sem,  how='left',
                                     left_on=['code_presentation', 'code_module'],
                                     right_on=['code_presentation', 'code_module'] )
table_1

,code_presentation,code_module,count_all_tries,count_succsess_tries
0,2013B,DDD,602,504
1,2013J,DDD,968,878
2,2014B,CCC,747,664
3,2014B,DDD,524,485
4,2014J,CCC,1168,1019
5,2014J,DDD,950,842


In [20]:
# Рассчитаем среднее количество дней, которое студенты тратят на сдачу экзамена по каждому из курсов в каждом семестре.
# Для этого выберем тип экзамена 'Exam' и количество баллов не менее 40 (score>=40). 
# Сгруппируем данные по семестру и  типу предмета и посчитаем среднее количество дней.
# Полученные результаты присоединим к предыдущей таблице по семестру и коду предмета

mean_time_sem = merge_studentAssessment_assessments.query("assessment_type=='Exam' and score>=40")\
                                    .groupby(['code_presentation','code_module'], as_index=False)\
                                    .agg({'date_submitted':'mean'}).rename(columns={'date_submitted':'mean_submit_time'})
mean_time_sem['mean_submit_time'] = round(mean_time['mean_submit_time'], 1)

all_sem_data = pd.merge(table_1, mean_time_sem,  how='left',
                                     left_on=['code_presentation', 'code_module'],
                                     right_on=['code_presentation', 'code_module'] )
all_sem_data

,code_presentation,code_module,count_all_tries,count_succsess_tries,mean_submit_time
0,2013B,DDD,602,504,239.3
1,2013J,DDD,968,878,238.0
2,2014B,CCC,747,664,NaN
3,2014B,DDD,524,485,NaN
4,2014J,CCC,1168,1019,NaN
5,2014J,DDD,950,842,NaN


In [21]:
# Создаю колонку, в которой рассчитывается завершаемость каждого предмета в семестре
# И вывожу семестр и предмет с самой низкой завершаемостью и самым долгим сроком сдачи

all_sem_data['completeness'] = round((100*all_sem_data.count_succsess_tries / all_sem_data.count_all_tries),2)
all_sem_data.sort_values('completeness').sort_values('mean_submit_time', ascending=False).head(1)

,code_presentation,code_module,count_all_tries,count_succsess_tries,mean_submit_time,completeness
0,2013B,DDD,602,504,239.3,83.72


In [22]:
# Задание 6.
#  Используя python, построй адаптированные RFM-кластеры студентов, чтобы качественно оценить свою аудиторию. 
# В адаптированной кластеризации можешь выбрать следующие метрики: R - среднее время сдачи одного экзамена, 
# F - завершаемость курсов, M - среднее количество баллов, получаемое за экзамен. 
# Подробно опиши, как ты создавал кластеры. Для каждого RFM-сегмента построй границы метрик 
# recency, frequency и monetary для интерпретации этих кластеров.

In [23]:
# Для каждого студента рассчитаем среднее время сдачи одного экзамена.

R_table = merge_studentAssessment_assessments.query("assessment_type=='Exam'")\
                                    .groupby(['id_student'],as_index=False)\
                                    .agg({'date_submitted':'mean'})\
                                    .rename(columns={'date_submitted':'Recency'})

In [24]:
# Также для каждого студента рассчитаем завершаемость курса

all_tries_stud = merge_studentAssessment_assessments.query("assessment_type =='Exam'")\
                                    .groupby(['id_student'], as_index=False)\
                                    .agg({'id_assessment':'count'})\
                                    .rename(columns={'id_assessment':'count_all_tries'})

succsess_stud = merge_studentAssessment_assessments.query("assessment_type =='Exam' and score >=40")\
                                    .groupby(['id_student'], as_index=False)\
                                    .agg({'id_assessment':'count'})\
                                    .rename(columns={'id_assessment':'count_succsess_tries'})

F_table = pd.merge(all_tries_stud, succsess_stud,  how='left',
                                     left_on=['id_student'],
                                     right_on=['id_student'] )
F_table['completeness'] = (100*F_table.count_succsess_tries / F_table.count_all_tries)
F_table = F_table.drop(columns=['count_all_tries', 'count_succsess_tries'])
F_table = F_table.rename(columns={'completeness':'Frequency'})


In [25]:
# Также для каждого студента рассчитаем среднее количество баллов, получаемое за экзамен

M_table = merge_studentAssessment_assessments.query("assessment_type=='Exam'")\
                                    .groupby(['id_student'],as_index=False)\
                                    .agg({'score':'mean'})\
                                    .rename(columns={'score':'Monetary'})

In [26]:
# Объединяем получившиеся таблицы

RF_table = pd.merge(R_table, F_table, how='left',
                           left_on=['id_student'],
                           right_on=['id_student'] )
RFM_table = pd.merge(RF_table, M_table, how='left',
                           left_on=['id_student'],
                           right_on=['id_student'] )
RFM_table

,id_student,Recency,Frequency,Monetary
0,23698,243.0,100.0,80.0
1,24213,236.0,100.0,58.0
2,27116,243.0,100.0,96.0
3,28046,237.0,100.0,40.0
4,28787,243.0,100.0,44.0
...,...,...,...,...
4628,2694886,236.0,100.0,69.0
4629,2694933,230.0,100.0,73.0
4630,2695608,237.0,100.0,73.0
4631,2697181,230.0,100.0,80.0


In [27]:
# Для дальнейшего деления студентов на группы, для каждого рассчитанного параметра
# ('Recency', 'Frequency', 'Monetary') рассчитаем различные квантили

quantiles = RFM_table[['Recency', 'Frequency', 'Monetary']].quantile([0.25, 0.5, 0.75]).to_dict()
quantiles

{'Recency': {0.25: 234.0, 0.5: 241.0, 0.75: 243.0},
 'Frequency': {0.25: 100.0, 0.5: 100.0, 0.75: 100.0},
 'Monetary': {0.25: 50.0, 0.5: 66.0, 0.75: 82.0}}

In [28]:
# Создаем функции для присвоения оценки от 1(худший) до 4 (лучший), чтобы применить к каждому рассчитанному параметру
# Для Recency (среднее время сдачи одного экзамена) наилучшим результатом является самое малое количество дней 
# (Recency <= 234.0 -> Оценка 4)
# Мы будем сравнивать значения Recency для каждого студента с получившимися значениями квантилей и на этой основе 
# давать оценку каждому студенту

def R_Class(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

# Поскольку Frequency (завершаемость курса) для всех студентов 100%, то мы сразу присваиваем им максимальную оценку успеха

def F_Class(x,p,d):
    return 4    

# Для параметра Monetary - среднее количество баллов, чем больше баллов, тем лучше оценка. Поэтому если у студента 
# значение параметра Monetary <= 50.0(рассчитанного квантиля (q=0.25)), то мы ему ставим минимальный балл 1. 
# Далее по этому принципу делим на группы 

def M_Class(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4        

In [29]:
# Создаем колоночки с оценками R, F, M для каждого студента, применив соответсвующую функцию
# Также создаем колоночку общей оценкой RFM, объединив записи из новых колонок

RFM_table['R'] = RFM_table['Recency'].apply(R_Class, args=('Recency',quantiles,))
RFM_table['F'] = RFM_table['Frequency'].apply(F_Class, args=('Frequency',quantiles,))
RFM_table['M'] = RFM_table['Monetary'].apply(M_Class, args=('Monetary',quantiles,))
RFM_table['RFM_Score'] = RFM_table.R.map(str)\
                        + RFM_table.F.map(str)\
                        + RFM_table.M.map(str)
RFM_table

,id_student,Recency,Frequency,Monetary,R,F,M,RFM_Score
0,23698,243.0,100.0,80.0,2,4,3,243
1,24213,236.0,100.0,58.0,3,4,2,342
2,27116,243.0,100.0,96.0,2,4,4,244
3,28046,237.0,100.0,40.0,3,4,1,341
4,28787,243.0,100.0,44.0,2,4,1,241
...,...,...,...,...,...,...,...,...
4628,2694886,236.0,100.0,69.0,3,4,3,343
4629,2694933,230.0,100.0,73.0,4,4,3,443
4630,2695608,237.0,100.0,73.0,3,4,3,343
4631,2697181,230.0,100.0,80.0,4,4,3,443


In [30]:
# ТОП-30 лучших студентов, успешно завершивших курс за минимальное количество дней с максимальным баллом

RFM_table[RFM_table['RFM_Score']=='444'].sort_values('Recency').sort_values('Monetary', ascending=False).head(30)

,id_student,Recency,Frequency,Monetary,R,F,M,RFM_Score
76,82575,230.0,100.0,100.0,4,4,4,444
2906,619303,230.0,100.0,100.0,4,4,4,444
1289,516726,230.0,100.0,100.0,4,4,4,444
2054,574909,234.0,100.0,100.0,4,4,4,444
1345,524128,230.0,100.0,100.0,4,4,4,444
181,155963,230.0,100.0,100.0,4,4,4,444
1962,571129,230.0,100.0,100.0,4,4,4,444
197,166160,230.0,100.0,100.0,4,4,4,444
1252,512475,230.0,100.0,100.0,4,4,4,444
1226,509183,230.0,100.0,100.0,4,4,4,444
